# Homework 1
**The goal of this homework is to improve the simple classifier explained in the book starting on page 40. It uses a pretrained transformer and then the logistic regression classifier of page 44 uses only the first hidden state for each input text.**

**You should use the full hidden states to classify each input text, and use a neural network instead of a regression classifier. Then, compare the results with the book’s on page 44.**

**You have 11 days to do it, by pairs. Start working early as you may need computing time to get the results. Please submit a Jupyter notebook or similar, and include some conclusions.**

## Solution

First, we make sure to install and import the necessary libraries

In [ ]:
%pip install -r requirements.txt

import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Then we set up `pytorch` to take profit of the GPU, if available, and load the dataset.

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
emotions_dataset = load_dataset("emotion")

Now, we use a pretrained model to transform the text into numerical representations. The model chosen was `distilbert-base-uncased`, as in the book.

In [28]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

# TODO: Add explanations for this section
A possible explanation to the more mystical parts can be simply "following the book" or something like that.

In [40]:
emotions_encoded = emotions_dataset.map(tokenize, batched=True, batch_size=None)
emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

print(emotions_encoded)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})


In [41]:
def compute_hidden_states(batch):
    inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}
        
emotions_hidden = emotions_encoded.map(compute_hidden_states, batched=True)

Map: 100%|██████████| 2000/2000 [00:29<00:00, 68.25 examples/s]


In [45]:
X_train = np.concatenate (
    (np.array(emotions_hidden["train"]["hidden_state"]), np.array(emotions_hidden["validation"]["hidden_state"]))
)
y_train = np.concatenate(
    (np.array(emotions_encoded["train"]["label"]), np.array(emotions_encoded["validation"]["label"]))
)

X_test = np.array(emotions_hidden["test"]["hidden_state"])
y_test = np.array(emotions_encoded["test"]["label"])

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(18000, 768) (18000,) (2000, 768) (2000,)


/tmp/ipykernel_411673/542799634.py:2: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  (np.array(emotions_hidden["train"]["hidden_state"]), np.array(emotions_hidden["validation"]["hidden_state"]))
/tmp/ipykernel_411673/542799634.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  (np.array(emotions_encoded["train"]["label"]), np.array(emotions_encoded["validation"]["label"]))
/tmp/ipykernel_411673/542799634.py:8: DeprecationWarning: __array__ implementation doesn't accept a co

Now we set up and train a feed-forward neural network to classify these numerical representations.

# TODO: actually do this